In [ ]:
# Filter out all lines and stops that are not relevant for the analysis

import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("lvb-spark") \
    .config('spark.master', 'local') \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.0') \
    .getOrCreate()

# Load departures data from MongoDB
df = spark.read.format("mongo") \
    .option("uri", "mongodb://mongo:27017/") \
    .option("database", "lvb") \
    .option("collection", "departures") \
    .load()

# Load relevant stops
with open('data/stops_fromRelevantLines.json', 'r') as f:
    relevant_stops = json.load(f)

# Load relevant lines
with open('data/relevantLines_with_stops.json', 'r') as f:
    relevant_lines = json.load(f)

# Get list of relevant stop IDs and line IDs
relevant_stop_ids = list(relevant_stops.keys())
relevant_line_ids = list(relevant_lines.keys())

# Filter departures by relevant stops
filtered_df = df.filter(col("stopId").isin(relevant_stop_ids))

# Count filtered stops
total_stops = df.select("stopId").distinct().count()
remaining_stops = filtered_df.select("stopId").distinct().count()
filtered_stops = total_stops - remaining_stops

print(f"Filtered out {filtered_stops} stops. {remaining_stops} stops remain.")

# Further filter by relevant lines
final_df = filtered_df.filter(col("lineId").isin(relevant_line_ids))

# Count filtered lines
total_lines = filtered_df.select("lineId").distinct().count()
remaining_lines = final_df.select("lineId").distinct().count()
filtered_lines = total_lines - remaining_lines

print(f"Filtered out {filtered_lines} lines. {remaining_lines} lines remain.")

# Output as Parquet
output_path = "data/filtered_departures.parquet"
final_df.write.mode("overwrite").parquet(output_path)

print(f"Filtered data saved to {output_path}")

# Show sample of the filtered data
final_df.show(5)

In [7]:
# Read the saved Parquet file
parquet_df = spark.read.parquet(output_path)

# Show sample of the filtered data from the Parquet file
print("Sample of data from the Parquet file:")
parquet_df.show(5)

# Optional: Display schema of the Parquet file
print("Schema of the Parquet file:")
parquet_df.printSchema()

# Optional: Get some basic statistics
print("Summary statistics:")
parquet_df.describe().show()


Sample of data from the Parquet file:
+---+--------------------+--------------------+-----+-------------------+-----------+-------------------+------+--------------------+-------------------+
|__v|                 _id|           crawlDate|delay|          direction|     lineId|        plannedWhen|stopId|              tripId|               when|
+---+--------------------+--------------------+-----+-------------------+-----------+-------------------+------+--------------------+-------------------+
|  0|{6583e768bf61bedc...|2023-12-21 07:21:...|   60|Technisches Rathaus|8-naslvt-12|2023-12-21 07:19:00|956126|1|1053955|11|81|2...|2023-12-21 07:20:00|
|  0|{6583e768bf61bedc...|2023-12-21 07:21:...|   60|       Messegelände|8-naslvt-16|2023-12-21 07:19:00|956291|1|293004|11|81|21...|2023-12-21 07:20:00|
|  0|{6583e768bf61bedc...|2023-12-21 07:21:...|    0|             Lößnig|8-naslvt-16|2023-12-21 07:20:00|956291|1|292994|10|81|21...|2023-12-21 07:20:00|
|  0|{6583e768bf61bedc...|2023-12-21 0

+-------+-------+-----------------+--------------------+---------+------------------+--------------------+
|summary|    __v|            delay|           direction|   lineId|            stopId|              tripId|
+-------+-------+-----------------+--------------------+---------+------------------+--------------------+
|  count|6923848|          6457767|             6923848|  6923848|           6923848|             6923848|
|   mean|    0.0|94.40022224400478|                null|     null|1759677.5941112514|                null|
| stddev|    0.0|313.0336486494021|                null|     null|2253638.2906197426|                null|
|    min|      0|                0|<+> Borna  über E...|3-bb-re10|            130636|1|1000001|0|81|16...|
|    max|      0|            30180|             fl.Text|      bus|            964749|1|999986|1|81|191...|
+-------+-------+-----------------+--------------------+---------+------------------+--------------------+



In [8]:
# Count total number of rows
total_rows = parquet_df.count()
mongo_count = spark.read.format("mongo") \
    .option("uri", "mongodb://mongo:27017/") \
    .option("database", "lvb") \
    .option("collection", "departures") \
    .load().count()

# Calculate the percentage of reduced data rows
reduced_percentage = ((mongo_count - total_rows) / mongo_count) * 100 if mongo_count > 0 else 0
print(f"Total number of rows: {total_rows}")
print(f"Percentage of reduced data rows: {reduced_percentage:.2f}%")

Total number of rows: 6923848
Percentage of reduced data rows: 9.80%
